# Assessment 2: Metric Learning with Oxford-IIIT Pet Dataset

## Introduction and Setup

This notebook implements a deep metric learning approach for the Oxford-IIIT Pet Dataset, focusing on learning an embedding space where similar pet breeds are close together and dissimilar ones are far apart. We'll explore different loss functions, evaluate the model on verification, retrieval, and few-shot classification tasks, and visualize the embedding space.

### Environment Setup and Package Installation

In [ ]:

# Check if running in Colab (to install dependencies and set up environment)
import sys
IN_COLAB = 'google.colab' in sys.modules

# Install required packages
if IN_COLAB:
    !pip install pytorch-metric-learning
    !pip install faiss-gpu
    !pip install umap-learn
    !pip install matplotlib seaborn scikit-learn tqdm
    !pip install gradio
    !pip install grad-cam

### Import Libraries

import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from collections import defaultdict
import itertools  # For generating pairs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
from torchvision import datasets, models, transforms
import torchvision.transforms.functional as TF

import pytorch_metric_learning
from pytorch_metric_learning import losses, miners, distances, reducers, testers

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
import umap

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")





## Data Loading and Preprocessing

In this section, we'll load the Oxford-IIIT Pet Dataset, perform necessary preprocessing, and create appropriate data loaders for our metric learning tasks.

In [ ]:

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load the dataset
def load_oxford_pets_dataset(root="./data", download=True):
    train_val_dataset = datasets.OxfordIIITPet(
        root=root, 
        split="trainval", 
        transform=train_transform, 
        download=download
    )
    
    test_dataset = datasets.OxfordIIITPet(
        root=root, 
        split="test", 
        transform=eval_transform, 
        download=download
    )
    
    # For evaluation, create a version of the training set with eval transforms
    eval_train_dataset = datasets.OxfordIIITPet(
        root=root, 
        split="trainval", 
        transform=eval_transform, 
        download=False
    )
    
    return train_val_dataset, test_dataset, eval_train_dataset



### Dataset Preparation for Different Tasks

In [ ]:

# Split data for training, validation and few-shot evaluation
def prepare_datasets(train_val_dataset, test_dataset, eval_train_dataset, num_holdout_classes=5, val_ratio=0.2):
    # Get the class names
    class_to_idx = train_val_dataset.class_to_idx
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    num_classes = len(class_to_idx)
    
    # Split classes for few-shot learning (hold out some classes for testing)
    all_class_indices = list(range(num_classes))
    holdout_class_indices = random.sample(all_class_indices, num_holdout_classes)
    training_class_indices = [i for i in all_class_indices if i not in holdout_class_indices]
    
    holdout_classes = [idx_to_class[i] for i in holdout_class_indices]
    print(f"Holdout classes for few-shot learning: {holdout_classes}")
    
    # Create datasets excluding holdout classes for main training
    train_val_indices = [i for i, (_, label) in enumerate(train_val_dataset) if label not in holdout_class_indices]
    test_indices = [i for i, (_, label) in enumerate(test_dataset) if label not in holdout_class_indices]
    eval_train_indices = [i for i, (_, label) in enumerate(eval_train_dataset) if label not in holdout_class_indices]
    
    # For few-shot learning, include only holdout classes
    few_shot_train_indices = [i for i, (_, label) in enumerate(train_val_dataset) if label in holdout_class_indices]
    few_shot_test_indices = [i for i, (_, label) in enumerate(test_dataset) if label in holdout_class_indices]
    
    # Split train/val
    train_indices, val_indices = train_test_split(
        train_val_indices, 
        test_size=val_ratio, 
        stratify=[train_val_dataset[i][1] for i in train_val_indices],
        random_state=42
    )
    
    # Create Subset datasets
    train_dataset = Subset(train_val_dataset, train_indices)
    val_dataset = Subset(train_val_dataset, val_indices)
    test_filtered_dataset = Subset(test_dataset, test_indices)
    eval_train_dataset = Subset(eval_train_dataset, eval_train_indices)
    
    # Create datasets for few-shot learning
    few_shot_train_dataset = Subset(train_val_dataset, few_shot_train_indices)
    few_shot_test_dataset = Subset(test_dataset, few_shot_test_indices)
    
    # Create dictionary for class mapping
    class_mapping = {
        'class_to_idx': class_to_idx,
        'idx_to_class': idx_to_class,
        'holdout_class_indices': holdout_class_indices,
        'training_class_indices': training_class_indices
    }
    
    return {
        'train': train_dataset,
        'val': val_dataset,
        'test': test_filtered_dataset,
        'eval_train': eval_train_dataset,
        'few_shot_train': few_shot_train_dataset,
        'few_shot_test': few_shot_test_dataset,
        'class_mapping': class_mapping
    }



### Create DataLoaders

In [ ]:

def create_dataloaders(datasets_dict, batch_size=32, num_workers=2):
    dataloaders = {}
    
    for key in ['train', 'val', 'test', 'eval_train', 'few_shot_train', 'few_shot_test']:
        if key == 'train':
            shuffle = True
        else:
            shuffle = False
            
        dataloaders[key] = DataLoader(
            datasets_dict[key],
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            pin_memory=True
        )
    
    return dataloaders




### Load and Prepare Data


In [ ]:

# Load the dataset
train_val_dataset, test_dataset, eval_train_dataset = load_oxford_pets_dataset()
print(f"Train+Val size: {len(train_val_dataset)}")
print(f"Test size: {len(test_dataset)}")

# Prepare datasets for different tasks
datasets_dict = prepare_datasets(train_val_dataset, test_dataset, eval_train_dataset)

# Create dataloaders
batch_size = 32  # Adjust based on your GPU/memory constraints
dataloaders = create_dataloaders(datasets_dict, batch_size=batch_size)

# Print dataset statistics
print("\nDataset Statistics:")
for key, dataloader in dataloaders.items():
    print(f"{key}: {len(dataloader.dataset)} samples")

class_mapping = datasets_dict['class_mapping']
num_classes = len(class_mapping['class_to_idx'])
print(f"Total number of classes: {num_classes}")
print(f"Number of training classes: {len(class_mapping['training_class_indices'])}")
print(f"Number of few-shot classes: {len(class_mapping['holdout_class_indices'])}")




## Model Architecture

In this section, we'll define our metric learning model architecture using a CNN backbone and a projection head.


In [ ]:

class EmbeddingNet(nn.Module):
    def __init__(self, backbone_name='resnet18', embedding_size=128, pretrained=True):
        super(EmbeddingNet, self).__init__()
        
        # Get backbone and its output size
        self.backbone, backbone_output_size = self._get_backbone(backbone_name, pretrained)
        
        # Projection head (MLP)
        self.projection_head = nn.Sequential(
            nn.Linear(backbone_output_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, embedding_size)
        )
        
    def _get_backbone(self, backbone_name, pretrained):
        """
        Create a backbone network from various architectures
        """
        if backbone_name == 'resnet18':
            backbone = models.resnet18(pretrained=pretrained)
            output_size = 512
        elif backbone_name == 'resnet34':
            backbone = models.resnet34(pretrained=pretrained)
            output_size = 512
        elif backbone_name == 'resnet50':
            backbone = models.resnet50(pretrained=pretrained)
            output_size = 2048
        elif backbone_name == 'efficientnet_b0':
            backbone = models.efficientnet_b0(pretrained=pretrained)
            output_size = 1280
        elif backbone_name == 'mobilenet_v2':
            backbone = models.mobilenet_v2(pretrained=pretrained)
            output_size = 1280
        elif backbone_name == 'densenet121':
            backbone = models.densenet121(pretrained=pretrained)
            output_size = 1024
        else:
            raise ValueError(f"Unsupported backbone: {backbone_name}")
        
        # For ResNet models
        if backbone_name.startswith('resnet'):
            # Remove the classification layer
            backbone = nn.Sequential(*list(backbone.children())[:-1])
        # For EfficientNet
        elif backbone_name.startswith('efficientnet'):
            backbone = nn.Sequential(*list(backbone.children())[:-1])
        # For MobileNet
        elif backbone_name.startswith('mobilenet'):
            backbone = nn.Sequential(*list(backbone.children())[:-1])
        # For DenseNet
        elif backbone_name.startswith('densenet'):
            backbone = nn.Sequential(
                backbone.features,
                nn.ReLU(inplace=True),
                nn.AdaptiveAvgPool2d((1, 1))
            )
        
        return backbone, output_size
        
    def forward(self, x):
        features = self.backbone(x)
        features = features.view(features.size(0), -1)
        embeddings = self.projection_head(features)
        
        # Normalize embeddings to unit length (important for cosine distance)
        normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
        return normalized_embeddings
    
    def get_embedding(self, x):
        return self.forward(x)


## Loss Function Implementation

Here we'll implement several loss functions for metric learning including Triplet Loss, Contrastive Loss, and ArcFace. We'll also implement miners for efficient training.


In [ ]:

def create_loss_and_miner(loss_type, margin=0.2, embedding_size=128, num_classes=32):
    """
    Create loss function and miner for metric learning
    """
    if loss_type == 'triplet':
        # Triplet loss with cosine distance
        distance = distances.CosineSimilarity()
        reducer = reducers.ThresholdReducer(low=0)
        loss_func = losses.TripletMarginLoss(margin=margin, distance=distance, reducer=reducer)
        mining_func = miners.TripletMarginMiner(margin=margin, distance=distance, type_of_triplets="semihard")
        
    elif loss_type == 'contrastive':
        # Contrastive loss
        distance = distances.CosineSimilarity()
        loss_func = losses.ContrastiveLoss(pos_margin=0.8, neg_margin=0.2, distance=distance)
        mining_func = miners.PairMarginMiner(pos_margin=0.8, neg_margin=0.2, distance=distance)
        
    elif loss_type == 'arcface':
        # ArcFace loss
        loss_func = losses.ArcFaceLoss(embedding_size, num_classes, margin=28.6, scale=64)
        mining_func = None
        
    else:
        raise ValueError(f"Unsupported loss type: {loss_type}")
        
    return loss_func, mining_func




### Hard Negative Mining (Bonus Implementation)

In [ ]:

class HardNegativePairMiner(miners.BaseMiner):
    def __init__(self, distance, neg_margin=0.2, hardest_fraction=0.5):
        super().__init__()
        self.distance = distance
        self.neg_margin = neg_margin
        self.hardest_fraction = hardest_fraction
        
    def mine(self, embeddings, labels, ref_emb=None, ref_labels=None):
        ref_emb, ref_labels = embeddings, labels
        dist_mat = self.distance(embeddings, ref_emb)
        
        # Get negative pairs (different classes)
        negative_mask = labels.unsqueeze(1) != ref_labels.unsqueeze(0)
        
        # For each anchor, find all negative pairs
        anchors, negatives = torch.where(negative_mask)
        
        if len(anchors) == 0:
            return empty_tensor(0), empty_tensor(0), empty_tensor(0), empty_tensor(0)
        
        # Get distances for all negative pairs
        distances = dist_mat[anchors, negatives]
        
        # Group by anchor
        anchor_groups = defaultdict(list)
        for i in range(len(anchors)):
            anchor_groups[anchors[i].item()].append((negatives[i].item(), distances[i].item()))
        
        # For each anchor, select the hardest negatives
        hard_a, hard_n = [], []
        for anchor, neg_dists in anchor_groups.items():
            # Sort negatives by distance (ascending for hardest cosine similarity)
            neg_dists.sort(key=lambda x: x[1], reverse=True)
            
            # Select hardest fraction
            num_to_select = max(1, int(len(neg_dists) * self.hardest_fraction))
            selected_negs = neg_dists[:num_to_select]
            
            for neg, dist in selected_negs:
                hard_a.append(anchor)
                hard_n.append(neg)
        
        return (
            torch.tensor(hard_a, device=embeddings.device), 
            empty_tensor(0), 
            empty_tensor(0), 
            torch.tensor(hard_n, device=embeddings.device)
        )
        
def empty_tensor(size):
    return torch.tensor([], device=device, dtype=torch.long).view(size)




## Hyperparameter Tuning Framework

This section implements a simple hyperparameter tuning framework to find the best combination of parameters for our model.

In [ ]:

# Add a simple hyperparameter tuning framework
def hyperparameter_tuning(param_grid, n_trials=5, evaluation_metric='combined'):
    """
    Simple hyperparameter tuning framework
    
    Args:
        param_grid (dict): Dictionary where keys are parameter names and values are lists of parameter values
        n_trials (int): Number of random combinations to try
        evaluation_metric (str): Which metric to use for choosing the best model ('roc_auc', 'recall', 'precision', 'combined')
    
    Returns:
        dict: Best parameters and the corresponding performance
    """
    print(f"Starting hyperparameter tuning with {n_trials} trials")
    print(f"Parameter grid: {param_grid}")
    
    # Load data (only once)
    train_val_dataset, test_dataset, eval_train_dataset = load_oxford_pets_dataset()
    datasets_dict = prepare_datasets(train_val_dataset, test_dataset, eval_train_dataset)
    
    # Track results
    results = []
    
    for trial in range(n_trials):
        # Sample random combination of hyperparameters
        params = {}
        for param_name, param_values in param_grid.items():
            params[param_name] = random.choice(param_values)
        
        print(f"\n\nTrial {trial+1}/{n_trials}")
        print(f"Parameters: {params}")
        
        # Create dataloaders
        dataloaders = create_dataloaders(
            datasets_dict, 
            batch_size=params.get('batch_size', 32),
            num_workers=params.get('num_workers', 2)
        )
        
        # Create model
        model = EmbeddingNet(
            backbone_name=params.get('backbone_name', 'resnet18'),
            embedding_size=params.get('embedding_size', 128),
            pretrained=True
        )
        model = model.to(device)
        
        # Create optimizer
        if params.get('optimizer_name', 'adam') == 'adam':
            optimizer = optim.Adam(
                model.parameters(), 
                lr=params.get('learning_rate', 1e-4),
                weight_decay=params.get('weight_decay', 0)
            )
        elif params.get('optimizer_name', 'adam') == 'sgd':
            optimizer = optim.SGD(
                model.parameters(), 
                lr=params.get('learning_rate', 1e-3),
                momentum=params.get('momentum', 0.9),
                weight_decay=params.get('weight_decay', 0)
            )
        
        # Create scheduler
        scheduler = None
        if params.get('use_scheduler', True):
            if params.get('scheduler_name', 'step') == 'step':
                scheduler = optim.lr_scheduler.StepLR(
                    optimizer, 
                    step_size=params.get('scheduler_step_size', 5), 
                    gamma=params.get('scheduler_gamma', 0.5)
                )
            elif params.get('scheduler_name', 'step') == 'cosine':
                scheduler = optim.lr_scheduler.CosineAnnealingLR(
                    optimizer,
                    T_max=params.get('num_epochs', 10),
                    eta_min=params.get('min_lr', 1e-6)
                )
        
        # Train model with fewer epochs for hyperparameter tuning
        num_epochs = params.get('num_epochs', 10)
        loss_type = params.get('loss_type', 'triplet')
        embedding_size = params.get('embedding_size', 128)
        
        try:
            # Train with minimal output
            model, history = train_model(
                model=model,
                dataloaders=dataloaders,
                loss_type=loss_type,
                optimizer=optimizer,
                scheduler=scheduler,
                num_epochs=num_epochs,
                embedding_size=embedding_size
            )
            
            # Simple evaluation to get a performance metric
            train_embeddings, train_labels = extract_embeddings(model, dataloaders['eval_train'])
            test_embeddings, test_labels = extract_embeddings(model, dataloaders['test'])
            
            # Verification Task
            verification_results = evaluate_verification(test_embeddings, test_labels)
            roc_auc = verification_results['roc_auc']
            
            # Retrieval Task
            retrieval_results = evaluate_retrieval(
                query_embeddings=test_embeddings,
                query_labels=test_labels,
                gallery_embeddings=train_embeddings,
                gallery_labels=train_labels,
                k_values=[1, 5]
            )
            recall_at_1 = retrieval_results['recall@1']
            precision_at_1 = retrieval_results['precision@1']
            
            # Calculate overall score based on the chosen evaluation metric
            if evaluation_metric == 'roc_auc':
                score = roc_auc
            elif evaluation_metric == 'recall':
                score = recall_at_1
            elif evaluation_metric == 'precision':
                score = precision_at_1
            else:  # combined
                score = 0.4 * roc_auc + 0.4 * recall_at_1 + 0.2 * precision_at_1
            
            # Store results
            results.append({
                'params': params,
                'roc_auc': roc_auc,
                'recall@1': recall_at_1,
                'precision@1': precision_at_1,
                'score': score,
                'val_loss': history['val_loss'][-1]
            })
            
            print(f"Trial {trial+1} results: ROC AUC = {roc_auc:.4f}, Recall@1 = {recall_at_1:.4f}, Score = {score:.4f}")
        
        except Exception as e:
            print(f"Error in trial {trial+1}: {e}")
            continue
    
    if not results:
        print("No successful trials!")
        return None
        
    # Find best parameters
    results.sort(key=lambda x: x['score'], reverse=True)
    best_result = results[0]
    
    print("\n\n============= Hyperparameter Tuning Results ==============")
    print(f"Best score: {best_result['score']:.4f}")
    print(f"Best parameters: {best_result['params']}")
    print(f"Performance: ROC AUC = {best_result['roc_auc']:.4f}, Recall@1 = {best_result['recall@1']:.4f}")
    print("===========================================================")
    
    # Plot validation loss curves for all trials
    if len(results) > 1:
        plt.figure(figsize=(12, 8))
        
        for i, result in enumerate(results):
            if 'val_loss' in result and len(result['val_loss']) > 0:
                plt.plot(range(1, len(result['val_loss'])+1), result['val_loss'], label=f"Trial {i+1} (score={result['score']:.4f})")
        
        plt.xlabel('Epoch')
        plt.ylabel('Validation Loss')
        plt.title('Hyperparameter Tuning - Validation Loss Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig('hyperparameter_tuning_results.png')
        plt.show()
    
    return best_result

# Example usage of the hyperparameter tuning framework
def run_hyperparameter_search():
    param_grid = {
        'backbone_name': ['resnet18', 'resnet34', 'efficientnet_b0', 'mobilenet_v2'],
        'embedding_size': [64, 128, 256],
        'batch_size': [16, 32, 64],
        'loss_type': ['triplet', 'contrastive', 'arcface'],
        'learning_rate': [1e-4, 5e-4, 1e-3],
        'optimizer_name': ['adam', 'sgd'],
        'weight_decay': [0, 1e-5, 1e-4],
        'scheduler_name': ['step', 'cosine'],
        'num_epochs': [10]  # Keep this fixed for faster tuning
    }
    
    # Run hyperparameter tuning
    best_params = hyperparameter_tuning(param_grid, n_trials=5, evaluation_metric='combined')
    return best_params

# For a more focused search with fewer parameters
def focused_hyperparameter_search():
    param_grid = {
        'backbone_name': ['resnet18'],
        'embedding_size': [128, 256],
        'batch_size': [32],
        'loss_type': ['triplet', 'contrastive'],
        'learning_rate': [1e-4, 5e-4],
        'optimizer_name': ['adam'],
        'scheduler_name': ['step'],
        'num_epochs': [10]
    }
    
    # Run focused hyperparameter tuning
    best_params = hyperparameter_tuning(param_grid, n_trials=4, evaluation_metric='combined')
    return best_params

# Uncomment to run hyperparameter tuning
# best_params = run_hyperparameter_search()
# best_params = focused_hyperparameter_search()



## Main Execution


In [ ]:

def evaluate_few_shot(support_embeddings, support_labels, query_embeddings, query_labels, n_way=5, k_shot=5, num_episodes=100):
    """
    Evaluate the model on n-way k-shot classification with confidence intervals
    
    Args:
        support_embeddings: Embeddings for the support set (training examples)
        support_labels: Labels for the support set
        query_embeddings: Embeddings for the query set (test examples)
        query_labels: Labels for the query set
        n_way: Number of classes in each episode
        k_shot: Number of examples per class in the support set
        num_episodes: Number of episodes to run for stable results
    
    Returns:
        dict: Dictionary containing accuracy statistics and confidence intervals
    """
    unique_labels = torch.unique(support_labels)
    if len(unique_labels) < n_way:
        print(f"Warning: Only {len(unique_labels)} classes available, but n_way={n_way}")
        n_way = len(unique_labels)
    
    accuracies = []
    
    # Progress bar for episodes
    progress_bar = tqdm(range(num_episodes), desc=f"{n_way}-way {k_shot}-shot evaluation")
    
    for episode in progress_bar:
        # Randomly select n classes for this episode
        selected_classes = np.random.choice(unique_labels.numpy(), n_way, replace=False)
        
        # Create support set (k examples per class)
        support_set_embeddings = []
        support_set_labels = []
        
        for class_idx, c in enumerate(selected_classes):
            # Get indices of examples of class c
            class_indices = torch.where(support_labels == c)[0]
            
            # Randomly select k examples
            if len(class_indices) >= k_shot:
                selected_indices = np.random.choice(class_indices.numpy(), k_shot, replace=False)
            else:
                # If not enough examples, use all and repeat some
                selected_indices = np.random.choice(class_indices.numpy(), k_shot, replace=True)
            
            for idx in selected_indices:
                support_set_embeddings.append(support_embeddings[idx])
                support_set_labels.append(class_idx)  # Use class index as the new label
        
        support_set_embeddings = torch.stack(support_set_embeddings)
        support_set_labels = torch.tensor(support_set_labels)
        
        # Create query set (all examples of the selected classes from the query set)
        query_set_indices = torch.tensor([i for i, label in enumerate(query_labels) if label in selected_classes])
        
        if len(query_set_indices) == 0:
            print("Warning: No query examples for selected classes")
            continue
            
        query_set_embeddings = query_embeddings[query_set_indices]
        query_set_labels = query_labels[query_set_indices]
        
        # Map original labels to new indices (0 to n_way-1)
        label_mapping = {selected_classes[i]: i for i in range(n_way)}
        query_set_labels = torch.tensor([label_mapping[label.item()] for label in query_set_labels])
        
        # Compute prototypes (mean embedding for each class)
        prototypes = torch.zeros(n_way, support_embeddings.size(1), device=support_embeddings.device)
        for c in range(n_way):
            prototypes[c] = support_set_embeddings[support_set_labels == c].mean(0)
        
        # Compute distances between query examples and prototypes
        # Using cosine similarity (higher means more similar)
        logits = torch.matmul(query_set_embeddings, prototypes.T)
        
        # Make predictions
        _, predictions = torch.max(logits, dim=1)
        
        # Compute accuracy
        accuracy = (predictions == query_set_labels).float().mean().item()
        accuracies.append(accuracy)
        
        # Update progress bar with current mean accuracy
        progress_bar.set_postfix({'mean_acc': np.mean(accuracies):.4f})
    
    # Calculate statistics
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    
    # Calculate 95% confidence interval using bootstrap
    bootstrap_samples = 1000
    bootstrap_means = []
    
    for _ in range(bootstrap_samples):
        # Sample with replacement from the accuracies
        bootstrap_sample = np.random.choice(accuracies, size=len(accuracies), replace=True)
        bootstrap_means.append(np.mean(bootstrap_sample))
    
    # Calculate 95% confidence interval
    conf_interval = np.percentile(bootstrap_means, [2.5, 97.5])
    
    print(f"{n_way}-way {k_shot}-shot classification:")
    print(f"  Mean accuracy: {mean_accuracy:.4f}")
    print(f"  Standard deviation: {std_accuracy:.4f}")
    print(f"  95% confidence interval: [{conf_interval[0]:.4f}, {conf_interval[1]:.4f}]")
    
    return {
        'mean_accuracy': mean_accuracy,
        'std_accuracy': std_accuracy,
        'conf_interval': conf_interval,
        'accuracies': accuracies
    }

def evaluate_multiple_few_shot_settings(support_embeddings, support_labels, query_embeddings, query_labels):
    """
    Evaluate few-shot learning across multiple n-way, k-shot configurations
    and visualize the results with error bars
    """
    # Define the settings to evaluate
    settings = [
        {'n_way': 5, 'k_shot': 1, 'num_episodes': 100},  # 5-way 1-shot
        {'n_way': 5, 'k_shot': 5, 'num_episodes': 100},  # 5-way 5-shot
        {'n_way': 10, 'k_shot': 1, 'num_episodes': 100}, # 10-way 1-shot
        {'n_way': 10, 'k_shot': 5, 'num_episodes': 100}  # 10-way 5-shot
    ]
    
    # Store results
    results = []
    
    # Run evaluation for each setting
    print("\n=== Few-Shot Learning Evaluation ===\n")
    for setting in settings:
        print(f"\nEvaluating {setting['n_way']}-way {setting['k_shot']}-shot learning...")
        result = evaluate_few_shot(
            support_embeddings=support_embeddings,
            support_labels=support_labels,
            query_embeddings=query_embeddings,
            query_labels=query_labels,
            n_way=setting['n_way'],
            k_shot=setting['k_shot'],
            num_episodes=setting['num_episodes']
        )
        
        results.append({
            'n_way': setting['n_way'],
            'k_shot': setting['k_shot'],
            'mean_accuracy': result['mean_accuracy'],
            'std_accuracy': result['std_accuracy'],
            'conf_interval': result['conf_interval']
        })
    
    # Create a visualization of the results with error bars
    plt.figure(figsize=(10, 6))
    
    # Group by n_way for bar chart
    n_way_values = sorted(list(set([r['n_way'] for r in results])))
    k_shot_values = sorted(list(set([r['k_shot'] for r in results])))
    
    # Set up bar positions
    bar_width = 0.35
    x = np.arange(len(n_way_values))
    
    # Plot bars for each k_shot value
    for i, k in enumerate(k_shot_values):
        means = []
        errors_lower = []
        errors_upper = []
        
        for n in n_way_values:
            # Find result for this n_way and k_shot
            result = next((r for r in results if r['n_way'] == n and r['k_shot'] == k), None)
            if result:
                means.append(result['mean_accuracy'])
                errors_lower.append(result['mean_accuracy'] - result['conf_interval'][0])
                errors_upper.append(result['conf_interval'][1] - result['mean_accuracy'])
            else:
                means.append(0)
                errors_lower.append(0)
                errors_upper.append(0)
        
        # Plot bars with error bars
        plt.bar(
            x + (i - 0.5*(len(k_shot_values)-1)) * bar_width, 
            means, 
            bar_width, 
            yerr=[errors_lower, errors_upper],
            label=f'{k}-shot',
            capsize=5
        )
    
    # Customize plot
    plt.xlabel('Number of Classes (N-way)')
    plt.ylabel('Accuracy')
    plt.title('Few-Shot Learning Performance')
    plt.xticks(x, [f'{n}-way' for n in n_way_values])
    plt.ylim(0, 1.0)
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add value labels on bars
    for i, result in enumerate(results):
        n_way_idx = n_way_values.index(result['n_way'])
        k_shot_idx = k_shot_values.index(result['k_shot'])
        x_pos = n_way_idx + (k_shot_idx - 0.5*(len(k_shot_values)-1)) * bar_width
        y_pos = result['mean_accuracy'] + 0.02
        plt.text(x_pos, y_pos, f"{result['mean_accuracy']:.3f}", 
                 ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('few_shot_evaluation.png', dpi=300)
    plt.show()
    
    return results


In [ ]:

def main(run_comprehensive_eval=True):
    # Uncomment to find the best parameters using hyperparameter tuning
    # best_params = focused_hyperparameter_search()
    # if best_params:
    #     backbone_name = best_params['params'].get('backbone_name', 'resnet18')
    #     embedding_size = best_params['params'].get('embedding_size', 128)
    #     batch_size = best_params['params'].get('batch_size', 32)
    #     loss_type = best_params['params'].get('loss_type', 'triplet')
    #     lr = best_params['params'].get('learning_rate', 1e-4)
    #     print(f"Using best parameters from hyperparameter tuning: {best_params['params']}")
    # else:
    #     # Use default parameters if hyperparameter tuning failed
    #     backbone_name = 'resnet18'
    #     embedding_size = 128
    #     batch_size = 32
    #     loss_type = 'triplet'
    #     lr = 1e-4
    
    # Model parameters (defaults)
    backbone_name = 'resnet18'  # Options: 'resnet18', 'resnet34', 'resnet50', 'efficientnet_b0', 'mobilenet_v2', 'densenet121'
    embedding_size = 128
    batch_size = 32  # Adjust based on your GPU
    num_workers = 2
    
    # Training parameters
    loss_type = 'triplet'  # Options: 'triplet', 'contrastive', 'arcface'
    num_epochs = 20
    lr = 1e-4

    # Load data
    train_val_dataset, test_dataset, eval_train_dataset = load_oxford_pets_dataset()
    datasets_dict = prepare_datasets(train_val_dataset, test_dataset, eval_train_dataset)
    dataloaders = create_dataloaders(datasets_dict, batch_size=batch_size, num_workers=num_workers)
    
    # Create model
    model = EmbeddingNet(backbone_name=backbone_name, embedding_size=embedding_size)
    model = model.to(device)
    
    # Create optimizer and scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    # Train model
    model, history = train_model(
        model=model,
        dataloaders=dataloaders,
        loss_type=loss_type,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=num_epochs,
        embedding_size=embedding_size
    )
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs+1), history['train_loss'], 'b-', label='Training Loss')
    plt.plot(range(1, num_epochs+1), history['val_loss'], 'r-', label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss ({loss_type} loss)')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_history.png')
    plt.show()
    
    # Save model
    torch.save({
        'model_state_dict': model.state_dict(),
        'embedding_size': embedding_size,
        'backbone_name': backbone_name,
        'class_mapping': datasets_dict['class_mapping']
    }, f'pet_metric_learning_{backbone_name}_{loss_type}.pth')
    
    if run_comprehensive_eval:
        # Run comprehensive evaluation
        print("\nRunning comprehensive model evaluation...")
        results_dir = f"./evaluation_results_{backbone_name}_{loss_type}"
        os.makedirs(results_dir, exist_ok=True)
        
        evaluation_results, eval_dir = comprehensive_evaluation(
            model=model,
            dataloaders=dataloaders,
            class_mapping=datasets_dict['class_mapping'],
            results_dir=results_dir
        )
        
        print(f"\nComprehensive evaluation completed. Results saved to {eval_dir}")
    else:
        # Extract embeddings for standard evaluation
        print("\nExtracting embeddings for evaluation...")
        train_embeddings, train_labels = extract_embeddings(model, dataloaders['eval_train'])
        test_embeddings, test_labels = extract_embeddings(model, dataloaders['test'])
        few_shot_train_embeddings, few_shot_train_labels = extract_embeddings(model, dataloaders['few_shot_train'])
        few_shot_test_embeddings, few_shot_test_labels = extract_embeddings(model, dataloaders['few_shot_test'])
        
        # Evaluation tasks
        print("\n1. Verification Task:")
        verification_results = evaluate_verification(test_embeddings, test_labels)
        
        print(f"\nVerification Results:")
        print(f"ROC AUC: {verification_results['roc_auc']:.4f}")
        print(f"Equal Error Rate (EER): {verification_results['eer']:.4f}")
        
        print("\n2. Retrieval Task:")
        retrieval_results = evaluate_retrieval(
            query_embeddings=test_embeddings,
            query_labels=test_labels,
            gallery_embeddings=train_embeddings,
            gallery_labels=train_labels,
            k_values=[1, 5, 10]
        )
        
        print("\n3. Few-shot Classification:")
        # Run comprehensive few-shot evaluation across multiple settings
        few_shot_results = evaluate_multiple_few_shot_settings(
            support_embeddings=few_shot_train_embeddings,
            support_labels=few_shot_train_labels,
            query_embeddings=few_shot_test_embeddings,
            query_labels=few_shot_test_labels
        )
        
        # Embedding visualization
        print("\n4. Embedding Visualization:")
        test_projection = visualize_embeddings(
            embeddings=test_embeddings,
            labels=test_labels,
            class_mapping=datasets_dict['class_mapping'],
            method='tsne',
            title='t-SNE Visualization of Test Embeddings'
        )
        
        # Visualize few-shot embeddings
        print("\nVisualizing few-shot embeddings:")
        # Combine few-shot train and test embeddings for visualization
        all_few_shot_embeddings = torch.cat([few_shot_train_embeddings, few_shot_test_embeddings], dim=0)
        all_few_shot_labels = torch.cat([few_shot_train_labels, few_shot_test_labels], dim=0)
        
        few_shot_projection = visualize_embeddings(
            embeddings=all_few_shot_embeddings,
            labels=all_few_shot_labels,
            class_mapping=datasets_dict['class_mapping'],
            method='tsne',
            title='t-SNE Visualization of Few-Shot Embeddings'
        )
        
        # Enhanced Grad-CAM Visualization
        print("\n5. Enhanced Grad-CAM Visualization:")
        visualize_grad_cam_comparisons(model, dataloaders['test'], datasets_dict['class_mapping'], num_comparisons=2)
        
        # Original Grad-CAM Visualization
        print("\n6. Individual Grad-CAM Visualization:")
        visualize_grad_cam(model, dataloaders['test'], datasets_dict['class_mapping'], num_images=3)
    
    print("\nEvaluation completed!")

if __name__ == "__main__":
    main(run_comprehensive_eval=True)  # Set to False for standard evaluation



## Bonus: Multiple Loss Function Comparison

In [ ]:

def compare_loss_functions():
    """
    Compare different loss functions for metric learning
    """
    # Model parameters
    backbone_name = 'resnet18'
    embedding_size = 128
    batch_size = 32
    num_workers = 2
    num_epochs = 15
    
    # Loss functions to compare
    loss_types = ['triplet', 'contrastive', 'arcface']
    
    # Load data (only once)
    train_val_dataset, test_dataset, eval_train_dataset = load_oxford_pets_dataset()
    datasets_dict = prepare_datasets(train_val_dataset, test_dataset, eval_train_dataset)
    dataloaders = create_dataloaders(datasets_dict, batch_size=batch_size, num_workers=num_workers)
    
    results = {}
    
    for loss_type in loss_types:
        print(f"\n{'=' * 40}")
        print(f"Training with {loss_type} loss")
        print(f"{'=' * 40}")
        
        # Create model
        model = EmbeddingNet(backbone_name=backbone_name, embedding_size=embedding_size)
        model = model.to(device)
        
        # Create optimizer and scheduler
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        
        # Train model
        model, history = train_model(
            model=model,
            dataloaders=dataloaders,
            loss_type=loss_type,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            embedding_size=embedding_size
        )
        
        # Extract embeddings for evaluation
        print("\nExtracting embeddings for evaluation...")
        train_embeddings, train_labels = extract_embeddings(model, dataloaders['eval_train'])
        test_embeddings, test_labels = extract_embeddings(model, dataloaders['test'])
        
        # Evaluation
        verification_results = evaluate_verification(test_embeddings, test_labels)
        
        retrieval_results = evaluate_retrieval(
            query_embeddings=test_embeddings,
            query_labels=test_labels,
            gallery_embeddings=train_embeddings,
            gallery_labels=train_labels,
            k_values=[1, 5]
        )
        
        # Store results
        results[loss_type] = {
            'verification': {
                'roc_auc': verification_results['roc_auc'],
                'eer': verification_results['eer']
            },
            'retrieval': {
                'recall@1': retrieval_results['recall@1'],
                'recall@5': retrieval_results['recall@5'],
                'precision@1': retrieval_results['precision@1'],
                'precision@5': retrieval_results['precision@5']
            }
        }
        
        # Save model
        torch.save({
            'model_state_dict': model.state_dict(),
            'embedding_size': embedding_size,
            'backbone_name': backbone_name,
            'class_mapping': datasets_dict['class_mapping']
        }, f'pet_metric_learning_{backbone_name}_{loss_type}_comparison.pth')
    
    # Compare results
    print("\n{'='*50}")
    print("Comparison of Loss Functions")
    print({'='*50})
    
    # Create a comparison table
    comparison_df = pd.DataFrame({
        'Loss Function': [],
        'ROC AUC': [],
        'EER': [],
        'Recall@1': [],
        'Recall@5': [],
        'Precision@1': [],
        'Precision@5': []
    })
    
    for loss_type, metrics in results.items():
        comparison_df = comparison_df.append({
            'Loss Function': loss_type,
            'ROC AUC': metrics['verification']['roc_auc'],
            'EER': metrics['verification']['eer'],
            'Recall@1': metrics['retrieval']['recall@1'],
            'Recall@5': metrics['retrieval']['recall@5'],
            'Precision@1': metrics['retrieval']['precision@1'],
            'Precision@5': metrics['retrieval']['precision@5']
        }, ignore_index=True)
    
    print(comparison_df)
    
    # Plot comparison
    plt.figure(figsize=(15, 10))
    
    metrics = ['ROC AUC', 'Recall@1', 'Recall@5', 'Precision@1', 'Precision@5']
    x = np.arange(len(metrics))
    width = 0.25
    
    for i, loss_type in enumerate(loss_types):
        values = [
            results[loss_type]['verification']['roc_auc'],
            results[loss_type]['retrieval']['recall@1'],
            results[loss_type]['retrieval']['recall@5'],
            results[loss_type]['retrieval']['precision@1'],
            results[loss_type]['retrieval']['precision@5']
        ]
        plt.bar(x + i*width, values, width, label=loss_type)
    
    plt.ylabel('Score')
    plt.title('Comparison of Loss Functions')
    plt.xticks(x + width, metrics)
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig('loss_function_comparison.png')
    plt.show()
    
    return results, comparison_df

# Uncomment to run the comparison
# loss_comparison_results, loss_comparison_df = compare_loss_functions()




## Bonus: Streamlit Demo

We've created a user-friendly Streamlit application that demonstrates the practical use of our trained metric learning model for pet similarity search. The application allows users to upload a pet image and find similar-looking pets based on the embeddings learned by our model.

### Running the Streamlit App

The complete Streamlit application code is available in the separate notebook file: `Charles_Watson_streamlit_pet_similarity_app.ipynb`.

To run the demo:

1. Ensure you have the necessary Python packages installed:
   ```bash
   pip install streamlit torch torchvision Pillow numpy tqdm matplotlib
   ```

2. Convert the Streamlit notebook to a Python script:
   ```bash
   jupyter nbconvert --to python Charles_Watson_streamlit_pet_similarity_app.ipynb
   ```

3. Make sure a trained model file (e.g., `pet_metric_learning_resnet18_triplet.pth`) exists in the working directory.

4. Run the Streamlit application:
   ```bash
   streamlit run Charles_Watson_streamlit_pet_similarity_app.py
   ```

### Features of the Streamlit App

- **Upload Interface**: Users can upload any pet image for similarity search
- **Configuration Options**: Customize model path, database location, and number of results
- **Interactive Results**: View similar pets with their breed labels and similarity scores
- **Database Management**: Load and reload the pet image database as needed

The app uses the same embedding model architecture (`EmbeddingNet`) as developed in this notebook (`Charles_Watson_Ndethi_Kibaki-Code.ipynb`), ensuring consistency between training and deployment.



## Conclusion

In this notebook, we have implemented a comprehensive metric learning pipeline for pet breed classification using the Oxford-IIIT Pet Dataset. We have:

1. Built a custom embedding model with a CNN backbone and projection head
2. Implemented various loss functions for metric learning (Triplet, Contrastive, ArcFace)
3. Developed evaluation methods for verification, retrieval, and few-shot classification
4. Created visualization tools for embedding spaces and feature importance (Grad-CAM)
5. Included bonus implementations for hard negative mining and loss function comparison
6. Moved the Streamlit demo to a separate `pet_similarity_app.py` file.

The code is modular and can be easily adapted for different settings and experimentation. To run the complete training and evaluation pipeline, simply execute the `main()` function.

## Advanced Model Evaluation Framework

This section implements a comprehensive evaluation framework that provides deeper insights into model performance across multiple dimensions: verification, retrieval, few-shot learning, and embedding quality. The framework generates detailed metrics and visualizations to understand strengths and weaknesses of the trained embeddings.

In [ ]:
import json
import os
from datetime import datetime
from sklearn.neighbors import NearestNeighbors

def comprehensive_evaluation(model, dataloaders, class_mapping, results_dir='./evaluation_results'):
    """
    Comprehensive evaluation of the model on all metrics
    
    Args:
        model: Trained model
        dataloaders: Dictionary of dataloaders ('train', 'val', 'test', etc.)
        class_mapping: Dictionary with class mapping information
        results_dir: Directory to save results
        
    Returns:
        Dictionary of all metrics
    """
    # Create results directory if it doesn't exist
    os.makedirs(results_dir, exist_ok=True)
    
    # Generate timestamp for uniqueness
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    eval_id = f"evaluation_{timestamp}"
    eval_dir = os.path.join(results_dir, eval_id)
    os.makedirs(eval_dir, exist_ok=True)
    
    model.eval()
    results = {}
    
    # Extract embeddings for all sets
    print("Extracting embeddings for evaluation...")
    all_embeddings = {}
    all_labels = {}
    
    for split_name in ['eval_train', 'test', 'few_shot_train', 'few_shot_test']:
        if split_name in dataloaders:
            print(f"  Processing {split_name} set...")
            embeddings, labels = extract_embeddings(model, dataloaders[split_name])
            all_embeddings[split_name] = embeddings
            all_labels[split_name] = labels
    
    # 1. Verification metrics
    print("\n1. Computing verification metrics...")
    verification_results = {}
    
    # Generate pairs for verification with more comprehensive settings
    test_pairs, test_pair_labels = generate_verification_pairs(
        all_embeddings['test'], all_labels['test'], num_pos_pairs=2000, num_neg_pairs=2000)
    
    # Compute ROC and other metrics
    fpr, tpr, thresholds = roc_curve(test_pair_labels, test_pairs)
    verification_results['roc_auc'] = auc(fpr, tpr)
    
    # Equal Error Rate (EER)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.absolute(fnr - fpr))
    eer_threshold = thresholds[eer_idx]
    eer = fpr[eer_idx]
    verification_results['eer'] = eer
    verification_results['eer_threshold'] = eer_threshold.item() if isinstance(eer_threshold, np.ndarray) else eer_threshold
    
    # F1 Score at different thresholds
    precision_values, recall_values, pr_thresholds = precision_recall_curve(test_pair_labels, test_pairs)
    f1_scores = 2 * precision_values * recall_values / (precision_values + recall_values + 1e-10)
    best_f1_idx = np.argmax(f1_scores)
    verification_results['best_f1_score'] = f1_scores[best_f1_idx]
    verification_results['best_f1_threshold'] = pr_thresholds[best_f1_idx] if best_f1_idx < len(pr_thresholds) else pr_thresholds[-1]
    
    # Plot ROC curve with more information
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC curve (AUC = {verification_results["roc_auc"]:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random')
    plt.plot([0, 1], [1, 0], 'g-.', linewidth=1.5, alpha=0.7, label='Perfect')
    plt.plot([eer, eer], [0, 1-eer], 'r:', linewidth=1.5)
    plt.plot([0, eer], [1-eer, 1-eer], 'r:', linewidth=1.5, label=f'EER = {eer:.4f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(eval_dir, 'roc_curve.png'), dpi=300)
    plt.close()
    
    # 2. Retrieval metrics with more detail
    print("\n2. Computing retrieval metrics...")
    retrieval_results = {}
    
    # More comprehensive retrieval evaluation
    query_embeddings = all_embeddings['test']
    query_labels = all_labels['test']
    gallery_embeddings = all_embeddings['eval_train']
    gallery_labels = all_labels['eval_train']
    
    # Compute pair-wise similarity matrix
    similarity_matrix = torch.matmul(query_embeddings, gallery_embeddings.T)
    
    # Evaluate at different k values
    k_values = [1, 5, 10, 20]
    for k in k_values:
        # Get top-k indices for each query
        _, indices = torch.topk(similarity_matrix, k=min(k, gallery_embeddings.size(0)), dim=1)
        
        # Compute metrics
        recall_at_k = 0
        precision_at_k = 0
        map_at_k = 0  # Mean Average Precision@K
        ndcg_at_k = 0  # Normalized Discounted Cumulative Gain@K
        
        for i, query_label in enumerate(query_labels):
            retrieved_labels = gallery_labels[indices[i]]
            relevant = (retrieved_labels == query_label).float()
            
            # Recall@K: How many of the relevant items are retrieved
            # (at least one relevant item in top-k)
            recall_at_k += (relevant.sum() > 0).float().item()
            
            # Precision@K: How many of the retrieved items are relevant
            precision_at_k += (relevant.sum() / k).item()
            
            # Average Precision calculation
            if relevant.sum() > 0:
                precision_at_ranks = torch.cumsum(relevant, dim=0) / torch.arange(1, k+1).float().to(relevant.device)
                ap = (precision_at_ranks * relevant).sum() / max(relevant.sum().item(), 1)
                map_at_k += ap.item()
            
            # NDCG calculation
            dcg = torch.sum(relevant / torch.log2(torch.arange(2, k+2).float().to(relevant.device)))
            # Ideal DCG would have all relevant items at the top
            ideal_relevant = torch.zeros_like(relevant)
            ideal_relevant[:min(int(relevant.sum().item()), k)] = 1
            idcg = torch.sum(ideal_relevant / torch.log2(torch.arange(2, k+2).float().to(relevant.device)))
            ndcg = dcg / max(idcg.item(), 1e-10)
            ndcg_at_k += ndcg.item()
        
        n_queries = len(query_labels)
        retrieval_results[f'recall@{k}'] = recall_at_k / n_queries
        retrieval_results[f'precision@{k}'] = precision_at_k / n_queries
        retrieval_results[f'map@{k}'] = map_at_k / n_queries
        retrieval_results[f'ndcg@{k}'] = ndcg_at_k / n_queries
        
        print(f"  Recall@{k}: {retrieval_results[f'recall@{k}']:.4f}")
        print(f"  Precision@{k}: {retrieval_results[f'precision@{k}']:.4f}")
        print(f"  MAP@{k}: {retrieval_results[f'map@{k}']:.4f}")
        print(f"  NDCG@{k}: {retrieval_results[f'ndcg@{k}']:.4f}")
    
    # Plot retrieval metrics across k values
    plt.figure(figsize=(12, 6))
    metrics = ['recall', 'precision', 'map', 'ndcg']
    metric_colors = {'recall': 'blue', 'precision': 'red', 'map': 'green', 'ndcg': 'purple'}
    
    for metric in metrics:
        values = [retrieval_results[f'{metric}@{k}'] for k in k_values]
        plt.plot(k_values, values, 'o-', label=f'{metric.upper()}@K', color=metric_colors[metric])
    
    plt.xlabel('K')
    plt.ylabel('Score')
    plt.title('Retrieval Metrics vs. K')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(eval_dir, 'retrieval_metrics.png'), dpi=300)
    plt.close()
    
    # 3. Few-shot learning with confidence intervals
    print("\n3. Computing few-shot learning metrics...")
    few_shot_results = {}
    
    if 'few_shot_train' in all_embeddings and 'few_shot_test' in all_embeddings:
        # Test different N-way K-shot configurations
        few_shot_configs = [
            {'n_way': 5, 'k_shot': 1, 'num_episodes': 100},
            {'n_way': 5, 'k_shot': 5, 'num_episodes': 100},
            {'n_way': 10, 'k_shot': 1, 'num_episodes': 100},
            {'n_way': 10, 'k_shot': 5, 'num_episodes': 100}
        ]
        
        for config in few_shot_configs:
            n_way = config['n_way']
            k_shot = config['k_shot']
            num_episodes = config['num_episodes']
            
            print(f"  Evaluating {n_way}-way {k_shot}-shot learning...")
            fs_result = evaluate_few_shot(
                all_embeddings['few_shot_train'],
                all_labels['few_shot_train'],
                all_embeddings['few_shot_test'],
                all_labels['few_shot_test'],
                n_way=n_way,
                k_shot=k_shot,
                num_episodes=num_episodes
            )
            few_shot_results[f'{n_way}way_{k_shot}shot'] = {
                'mean_accuracy': fs_result['mean_accuracy'],
                'std_accuracy': fs_result['std_accuracy'],
                'conf_interval': [float(x) for x in fs_result['conf_interval']]
            }
    else:
        print("  Few-shot datasets not available. Skipping few-shot evaluation.")
    
    # 4. Embedding quality analysis
    print("\n4. Analyzing embedding quality...")
    embedding_quality = {}
    
    # Intra-class and inter-class distances
    test_embeddings = all_embeddings['test']
    test_labels = all_labels['test']
    intra_class_dists = []
    inter_class_dists = []
    
    # Calculate pairwise distances
    for i in range(len(test_embeddings)):
        for j in range(i+1, len(test_embeddings)):
            # Cosine distance (1 - cosine similarity)
            similarity = torch.dot(test_embeddings[i], test_embeddings[j])
            distance = 1.0 - similarity.item()
            
            if test_labels[i] == test_labels[j]:
                intra_class_dists.append(distance)
            else:
                inter_class_dists.append(distance)
    
    intra_class_dists = np.array(intra_class_dists)
    inter_class_dists = np.array(inter_class_dists)
    
    # Calculate statistics
    embedding_quality['intra_class_mean'] = float(np.mean(intra_class_dists))
    embedding_quality['intra_class_std'] = float(np.std(intra_class_dists))
    embedding_quality['inter_class_mean'] = float(np.mean(inter_class_dists))
    embedding_quality['inter_class_std'] = float(np.std(inter_class_dists))
    
    # Fisher criterion (larger is better separation)
    intra_var = np.var(intra_class_dists)
    inter_var = np.var(inter_class_dists)
    intra_mean = np.mean(intra_class_dists)
    inter_mean = np.mean(inter_class_dists)
    fisher_score = ((inter_mean - intra_mean) ** 2) / (intra_var + inter_var)
    embedding_quality['fisher_score'] = float(fisher_score)
    
    print(f"  Intra-class distance: {embedding_quality['intra_class_mean']:.4f} ± {embedding_quality['intra_class_std']:.4f}")
    print(f"  Inter-class distance: {embedding_quality['inter_class_mean']:.4f} ± {embedding_quality['inter_class_std']:.4f}")
    print(f"  Fisher score: {embedding_quality['fisher_score']:.4f}")
    
    # Plot distance distributions
    plt.figure(figsize=(10, 6))
    plt.hist(intra_class_dists, bins=50, alpha=0.6, label='Intra-class', density=True)
    plt.hist(inter_class_dists, bins=50, alpha=0.6, label='Inter-class', density=True)
    plt.xlabel('Cosine Distance')
    plt.ylabel('Density')
    plt.title('Distribution of Intra-class vs Inter-class Distances')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(eval_dir, 'distance_distributions.png'), dpi=300)
    plt.close()
    
    # 5. Confusion matrix of most confused classes
    print("\n5. Identifying commonly confused classes...")
    # Create a confusion dictionary between classes
    confusion_dict = {i: {} for i in range(len(np.unique(test_labels)))}
    
    # For each query, see what other classes it's closest to
    class_counts = np.bincount(test_labels.numpy())
    unique_classes = np.unique(test_labels.numpy())
    
    # Create a matrix to track confusion between classes
    confusion_matrix = torch.zeros(len(unique_classes), len(unique_classes))
    
    # Calculate confusion based on nearest neighbors
    for i, query_emb in enumerate(test_embeddings):
        query_label = test_labels[i].item()
        
        # Calculate similarities to all other embeddings
        similarities = torch.matmul(query_emb.unsqueeze(0), test_embeddings.T).squeeze()
        
        # Remove self-similarity
        similarities[i] = -1.0
        
        # Get top 5 most similar
        topk_values, topk_indices = torch.topk(similarities, k=min(5, len(similarities)-1))
        
        # Update confusion matrix
        for idx in topk_indices:
            other_label = test_labels[idx].item()
            if query_label != other_label:
                confusion_matrix[query_label, other_label] += 1
    
    # Normalize confusion matrix by class frequency
    for i in range(confusion_matrix.size(0)):
        if class_counts[i] > 0:
            confusion_matrix[i] /= class_counts[i]
    
    # Convert class indices to class names
    idx_to_class = class_mapping['idx_to_class']
    class_names = [idx_to_class[i] for i in unique_classes]
    
    # Get most confused pairs
    confused_pairs = []
    n_classes = confusion_matrix.size(0)
    flattened = confusion_matrix.view(-1)
    topk_values, topk_indices = torch.topk(flattened, k=min(10, n_classes*n_classes))
    
    for idx in topk_indices:
        if topk_values[idx] > 0:
            i = idx.item() // n_classes
            j = idx.item() % n_classes
            class1 = idx_to_class[i]
            class2 = idx_to_class[j]
            confused_pairs.append((class1, class2, topk_values[idx].item()))
    
    # Display most confused class pairs
    print("  Most confused class pairs:")
    for class1, class2, score in confused_pairs[:5]:
        print(f"    {class1} confused with {class2}: {score:.4f}")
    
    # Combine all results
    all_results = {
        'verification': verification_results,
        'retrieval': retrieval_results,
        'few_shot': few_shot_results,
        'embedding_quality': embedding_quality
    }
    
    # Save results as JSON
    with open(os.path.join(eval_dir, 'evaluation_results.json'), 'w') as f:
        json.dump(all_results, f, indent=4)
    
    print(f"\nEvaluation complete. Results saved to {eval_dir}")
    
    return all_results, eval_dir

# Helper function: Generate verification pairs with better sampling
def generate_verification_pairs(embeddings, labels, num_pos_pairs=1000, num_neg_pairs=1000):
    """
    Generate balanced positive and negative pairs for verification task, with better sampling
    
    Args:
        embeddings: Embedding tensor
        labels: Labels tensor
        num_pos_pairs: Number of positive pairs to generate
        num_neg_pairs: Number of negative pairs to generate
        
    Returns:
        Tuple of (similarity scores, pair labels)
    """
    # Convert to numpy for easier indexing if they're not already
    if isinstance(embeddings, torch.Tensor):
        embeddings_np = embeddings.cpu().numpy()
    else:
        embeddings_np = embeddings
        
    if isinstance(labels, torch.Tensor):
        labels_np = labels.cpu().numpy()
    else:
        labels_np = labels
    
    unique_labels = np.unique(labels_np)
    pairs = []
    pair_labels = []
    
    # Generate positive pairs (same class) with stratified sampling
    pos_pair_count = 0
    pairs_per_class = max(1, num_pos_pairs // len(unique_labels))
    
    # First try to sample evenly from each class
    for label in unique_labels:
        indices = np.where(labels_np == label)[0]
        if len(indices) < 2:
            continue
            
        # Calculate how many pairs we can generate from this class
        max_pairs = min(pairs_per_class, len(indices) * (len(indices) - 1) // 2)
        
        # Sample unique pairs
        pair_count = 0
        max_attempts = max_pairs * 3  # Allow some failed attempts
        attempts = 0
        
        while pair_count < max_pairs and attempts < max_attempts:
            idx1, idx2 = np.random.choice(indices, 2, replace=False)
            
            # Skip if pair already exists
            if any((p[0] == idx1 and p[1] == idx2) or (p[0] == idx2 and p[1] == idx1) for p in pairs):
                attempts += 1
                continue
                
            # Use cosine similarity since our embeddings are normalized
            similarity = np.dot(embeddings_np[idx1], embeddings_np[idx2])
            
            pairs.append((idx1, idx2))
            pair_labels.append(1)  # 1 for same class
            pairs.append(similarity)
            pos_pair_count += 1
            pair_count += 1
        
    # If we need more positive pairs, sample randomly
    if pos_pair_count < num_pos_pairs:
        attempts = 0
        max_attempts = (num_pos_pairs - pos_pair_count) * 5
        
        while pos_pair_count < num_pos_pairs and attempts < max_attempts:
            label = np.random.choice(unique_labels)
            indices = np.where(labels_np == label)[0]
            if len(indices) < 2:
                attempts += 1
                continue
                
            idx1, idx2 = np.random.choice(indices, 2, replace=False)
            
            # Skip if pair already exists
            if any((p[0] == idx1 and p[1] == idx2) or (p[0] == idx2 and p[1] == idx1) for p in pairs):
                attempts += 1
                continue
                
            similarity = np.dot(embeddings_np[idx1], embeddings_np[idx2])
            
            pairs.append((idx1, idx2))
            pair_labels.append(1)
            pairs.append(similarity)
            pos_pair_count += 1
    
    # Generate negative pairs (different classes)
    # Use a mix of random and hard negative mining
    neg_pair_count = 0
    
    # First, compute all-to-all similarities for more efficient hard negative mining
    sim_matrix = np.zeros((len(embeddings_np), len(embeddings_np)))
    for i in range(len(embeddings_np)):
        sim_matrix[i] = np.dot(embeddings_np, embeddings_np[i])
    
    # Generate some truly random negatives
    random_neg_pairs = num_neg_pairs // 2
    attempts = 0
    max_attempts = random_neg_pairs * 5
    
    while neg_pair_count < random_neg_pairs and attempts < max_attempts:
        idx1 = np.random.randint(0, len(labels_np))
        idx2 = np.random.randint(0, len(labels_np))
        
        # Ensure different classes
        if labels_np[idx1] == labels_np[idx2]:
            attempts += 1
            continue
            
        # Skip if pair already exists
        if any((p[0] == idx1 and p[1] == idx2) or (p[0] == idx2 and p[1] == idx1) for p in pairs):
            attempts += 1
            continue
            
        similarity = sim_matrix[idx1, idx2]
        
        pairs.append((idx1, idx2))
        pair_labels.append(0)  # 0 for different class
        pairs.append(similarity)
        neg_pair_count += 1
    
    # Now generate hard negatives (different classes but high similarity)
    hard_neg_pairs = num_neg_pairs - neg_pair_count
    
    # For each class, find most similar examples from other classes
    for label in unique_labels:
        if neg_pair_count >= num_neg_pairs:
            break
            
        # Get examples of this class
        class_indices = np.where(labels_np == label)[0]
        if len(class_indices) == 0:
            continue
            
        # For each example, find most similar from other classes
        for idx1 in class_indices:
            if neg_pair_count >= num_neg_pairs:
                break
                
            # Get similarities to all other examples
            similarities = sim_matrix[idx1]
            
            # Create a mask for examples from different classes
            diff_class_mask = (labels_np != label)
            
            # Get most similar example from a different class
            diff_class_sims = similarities * diff_class_mask
            if np.max(diff_class_sims) == 0:  # No valid candidates
                continue
                
            idx2 = np.argmax(diff_class_sims)
            similarity = similarities[idx2]
            
            # Skip if pair already exists
            if any((p[0] == idx1 and p[1] == idx2) or (p[0] == idx2 and p[1] == idx1) for p in pairs):
                continue
                
            pairs.append((idx1, idx2))
            pair_labels.append(0)  # 0 for different class
            pairs.append(similarity)
            neg_pair_count += 1
    
    similarities = [pairs[i*2+1] for i in range(len(pair_labels))]
    return np.array(similarities), np.array(pair_labels)

## Enhanced Main Execution

The updated main function includes a call to our comprehensive evaluation framework to generate detailed metrics.